# Automatic Sandpit Refinement for D-Flow FM

This notebook provides an automated workflow for refining unstructured grids around sandpit areas in D-Flow FM models. The refinement process uses Casulli refinement to gradually transition from coarse background resolution to fine target resolution.

## Workflow Overview

1. **Configuration**: Set file paths and refinement parameters
2. **Load Grid and Polygons**: Load the grid and create/load sandpit polygons
3. **Plan Refinement**: Analyze grid resolution and generate refinement zones
4. **Execute Refinement**: Apply Casulli refinement to the grid
5. **Monitor Quality** (Optional): Analyze grid quality metrics

## Requirements

- `dfm_tools`
- `meshkernel`
- `numpy`
- `matplotlib`
- `shapely`

In [ ]:
# Step 0: Configuration and Setup

import numpy as np
import os
import sys
import dfm_tools as dfmt
import matplotlib.pyplot as plt

# Add project root to Python path (for local installations)
project_root = os.path.abspath(os.path.join(os.getcwd(), '..'))
if project_root not in sys.path:
    sys.path.insert(0, project_root)

# Import utility functions
from src.polygon_utils import InteractivePolygonDrawer, load_pol_file, save_pol_file, generate_refinement_polygons, expand_polygon_outward
from src.refinement_utils import compute_refinement_steps, apply_casulli_refinement, print_refinement_summary
from src.visualization_utils import plot_grid, set_interactive_plots, set_static_plots, is_codespace
from src.monitoring_utils import analyze_grid_quality, plot_grid_quality

# Base path configuration - works both locally and in Codespace
nc_file = 'dcsm_0_125nm_2ref_bathygr7_RGFGRID_net.nc'
nc_path = os.path.join('data', 'input', nc_file)

# Output settings
output_dir = os.path.join('data', 'output')
os.makedirs(output_dir, exist_ok=True)

# User configuration
use_existing_pol_file = True
existing_pol_file = os.path.join('data', 'input', 'sandpits.pol')

# Refinement parameters
target_resolution = 30  # Target resolution in meters for the finest refinement level
buffer_around_sandpit = 250  # Buffer around sandpit polygons in meters
N = 7  # Number of transition cells

# Environment detection
if is_codespace():
    print("🔧 Running in GitHub Codespace")
else:
    print("🔧 Running locally")

print("✅ Configuration loaded")
print(f"   Target: {target_resolution}m | Buffer: {buffer_around_sandpit}m | Transitions: {N}")

## Step 1: Load Grid and Define Sandpit Polygons

This step loads the D-Flow FM grid and either:
- Loads existing sandpit polygons from a .pol file
- Opens an interactive drawing tool to create new polygons

The polygons define the areas where refinement will be applied.

**Note**: Interactive polygon drawing is only available when running locally, not in Codespace.

In [ ]:
# Step 1: Load Grid and Create/Load Polygons

# Check if NetCDF file exists
if not os.path.exists(nc_path):
    print(f"❌ NetCDF file not found: {nc_path}")
    if is_codespace():
        print("   File should be automatically available in Codespace")
    else:
        print("   Please download the file from: https://github.com/your-repo/releases")
        print("   Or use git lfs pull to download LFS files")
    raise FileNotFoundError(f"Required file not found: {nc_path}")

# Load grid data
print("📊 Loading grid...")
ugrid = dfmt.open_partitioned_dataset(nc_path)
ugrid_original = dfmt.open_partitioned_dataset(nc_path)  # Keep original for later comparison

# Convert to meshkernel objects for consistent plotting and refinement
mk_object = ugrid.grid.meshkernel
mk_backup = ugrid_original.grid.meshkernel  # Keep backup

print(f"✅ Grid loaded: {ugrid.sizes['nmesh2d_node']} nodes, {ugrid.sizes['nmesh2d_face']} faces")

# Handle polygon input/creation
if use_existing_pol_file:
    # Load existing polygon file
    if os.path.exists(existing_pol_file):
        polygons = load_pol_file(existing_pol_file)
        print(f"✅ Loaded {len(polygons)} sandpit polygons from file")
    else:
        print(f"❌ File {existing_pol_file} not found, switching to interactive mode")
        use_existing_pol_file = False

if not use_existing_pol_file:
    # Interactive polygon creation
    if is_codespace():
        print("❌ Interactive polygon drawing not available in Codespace")
        print("   Please create a sandpits.pol file or run locally for interactive drawing")
        raise RuntimeError("Interactive mode not supported in Codespace")
    
    set_interactive_plots()
    print("🖱️  Opening interactive polygon drawing tool...")
    print("   Instructions: RIGHT CLICK → add vertex | ENTER → finish polygon | Close window when done")
    
    drawer = InteractivePolygonDrawer(ugrid, nc_path)
    polygons = drawer.draw_polygons()
    
    if polygons:
        output_file = os.path.join(output_dir, 'sandpits.pol')
        save_pol_file(polygons, output_file)
        print(f"✅ Saved {len(polygons)} polygons to {output_file}")

## Step 2: Plan Refinement Strategy

This step:
1. Analyzes the current grid resolution within the sandpit polygons
2. Calculates the number of refinement steps needed
3. Generates refinement zones with automatic overlap detection and merging
4. Visualizes the refinement plan

The refinement zones are created from coarse (outer) to fine (inner) resolution.

In [ ]:
# Step 2: Plan Refinement Strategy

# Analyze grid resolution and compute refinement steps
print("🔍 Analyzing grid resolution...")
refinement_params = compute_refinement_steps(ugrid, target_resolution, polygons)

# Generate refinement polygons with overlap merging
print("📐 Generating refinement zones...")
(all_refinement_polygons, all_original_polygons, 
 buffer_polygons, expansions) = generate_refinement_polygons(
    polygons, refinement_params, buffer_around_sandpit, N)

# Store original buffer polygons for visualization
original_buffer_polygons = []
for i, polygon in enumerate(polygons):
    polygon_array = np.array(polygon)
    center_lat = np.mean(polygon_array[:, 1])
    expanded_polygon = expand_polygon_outward(polygon, buffer_around_sandpit, center_lat)
    original_buffer_polygons.append(expanded_polygon)

# Visualize refinement plan
print("📈 Visualizing refinement plan...")
set_static_plots()
plot_grid(mk_object, polygons, all_refinement_polygons, all_original_polygons,
          buffer_polygons, refinement_params['envelope_sizes_m'], refinement_params['n_steps'],
          original_buffer_polygons=original_buffer_polygons,
          title='Refinement Plan: Sandpit Polygons and Merged Refinement Zones')
plt.show()

## Step 3: Execute Grid Refinement

This step applies Casulli refinement to the meshkernel object using the generated refinement zones. The refinement is applied from outside to inside (coarse to fine) to ensure smooth transitions.

After refinement, the refined grid is visualized showing the final mesh structure.

In [ ]:
# Step 3: Execute Grid Refinement

# Apply Casulli refinement
print("⚙️  Executing refinement...")
apply_casulli_refinement(mk_object, all_refinement_polygons)

# Visualize refined grid
print("📊 Visualizing refined grid...")
set_static_plots()
plot_grid(mk_object, polygons, all_refinement_polygons, all_original_polygons,
          buffer_polygons, refinement_params['envelope_sizes_m'], refinement_params['n_steps'],
          title='Refined Grid: Final Result with Casulli Refinement')

# Print refinement summary
print_refinement_summary(polygons, all_refinement_polygons, 
                        refinement_params['envelope_sizes_m'], 
                        refinement_params['n_steps'], buffer_polygons)

# Save refined grid if needed
# ugrid_refined = dfmt.meshkernel_to_UgridDataset(mk_object, crs='EPSG:4326')
# output_nc = os.path.join(output_dir, 'refined_grid.nc')
# ugrid_refined.to_netcdf(output_nc)
# print(f"💾 Refined grid saved to: {output_nc}")

## Step 4: Monitor Grid Quality (Optional)

This optional step analyzes the quality of the refined grid by examining:
- **Resolution**: Face areas converted to characteristic lengths
- **Smoothness**: Ratio of adjacent cell sizes (target < 1.4)
- **Orthogonality**: Deviation from 90° angles (target < 0.01)

**Note**: This analysis can be computationally intensive for large grids, especially in Codespace.

In [ ]:
# Step 4: (Optional) Monitor Grid Quality

if is_codespace():
    print("⚠️  Grid quality analysis may be slow in Codespace environment")
    print("   Consider running this step locally for better performance")

print("🔬 Analyzing grid quality metrics...")
quality_data = analyze_grid_quality(mk_object, ugrid_original, all_refinement_polygons, polygons)

print("📊 Creating quality visualization...")
set_static_plots()
plot_grid_quality(quality_data, all_refinement_polygons, target_resolution)

## Additional Operations (Optional)

Here are some additional operations you might want to perform after refinement:

In [ ]:
# Optional: Export refined grid to NetCDF
save_refined_grid = False  # Set to True if you want to save

if save_refined_grid:
    print("💾 Saving refined grid...")
    ugrid_refined = dfmt.meshkernel_to_UgridDataset(mk_object, crs='EPSG:4326')
    output_nc = os.path.join(output_dir, 'refined_grid.nc')
    ugrid_refined.to_netcdf(output_nc)
    print(f"✅ Refined grid saved to: {output_nc}")